In [2]:
!pip install -U transformers

In [3]:
# Update libraries
!pip install --upgrade transformers datasets evaluate

# Restart the kernel after running this cell and then re-run all cells

# Add this before loading the dataset
import os
cache_dir = "./hf_cache"
os.makedirs(cache_dir, exist_ok=True)

# Then load the dataset with the cache directory specified
from datasets import load_dataset
dataset = load_dataset("imdb", cache_dir=cache_dir)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [4]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
import torch


In [5]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [6]:
def tokenize_function(examples):
    return tokenizer(examples["text"],truncation=True, padding='max_length',max_length=256)

train_dataset = dataset['train'].map(tokenize_function, batched=True)
test_dataset = dataset['test'].map(tokenize_function, batched=True)

train_dataset.set_format(type='torch', columns = ['input_ids', 'attention_mask', 'label'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

In [7]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
training_args = TrainingArguments(
    output_dir="./bert_imdb_output",
    #evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
    save_strategy="no",
    #load_best_model_at_end=True,
)

In [12]:
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = test_dataset,
    tokenizer = tokenizer
)



<ipython-input-12-fffbc65384da>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [13]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: mj43467 (mj43467-student) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
50,0.664700
100,0.509300
150,0.410000
200,0.338400
250,0.406200
300,0.332500
350,0.321500
400,0.347000
450,0.328700
500,0.374000


TrainOutput(global_step=6250, training_loss=0.24666770393371582, metrics={'train_runtime': 2576.5633, 'train_samples_per_second': 19.406, 'train_steps_per_second': 2.426, 'total_flos': 6577776384000000.0, 'train_loss': 0.24666770393371582, 'epoch': 2.0})

In [14]:
trainer.evaluate()

{'eval_loss': 0.3184192180633545,
 'eval_runtime': 379.5742,
 'eval_samples_per_second': 65.863,
 'eval_steps_per_second': 8.233,
 'epoch': 2.0}

In [15]:
import evaluate
import numpy as np

accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return accuracy.compute(predictions=predictions, references=labels)

In [16]:
from sklearn.metrics import accuracy_score

# Get predictions
predictions = trainer.predict(test_dataset)
preds = predictions.predictions.argmax(-1)
labels = predictions.label_ids

# Accuracy
accuracy = accuracy_score(labels, preds)
print(f"Test Accuracy: {accuracy:.4f}")

Test Accuracy: 0.9238


In [17]:
from sklearn.metrics import precision_recall_fscore_support
precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

Precision: 0.9151
Recall: 0.9342
F1 Score: 0.9246


In [18]:
model.save_pretrained("./bert_finetuned_imdb")
tokenizer.save_pretrained("./bert_finetuned_imdb")

('./bert_finetuned_imdb/tokenizer_config.json',
 './bert_finetuned_imdb/special_tokens_map.json',
 './bert_finetuned_imdb/vocab.txt',
 './bert_finetuned_imdb/added_tokens.json')

In [ ]:
# to load it again in future
from transformers import BertForSequenceClassification, BertTokenizer

model = BertForSequenceClassification.from_pretrained("./bert_finetuned_imdb")
tokenizer = BertTokenizer.from_pretrained("./bert_finetuned_imdb")
